In [2]:
# Python script to thoroughly check heterozygosity per chromosome from a VCF file

import pandas as pd

# Path to your VCF file
vcf_file = 'snps_clean.vcf'


In [3]:
vcf_file

'snps_clean.vcf'

In [4]:
# Parsing the VCF file
data = []
with open(vcf_file, 'r') as file:
    for line in file:
        if not line.startswith('#'):
            parts = line.strip().split('\t')
            chrom = parts[0]
            genotype = parts[9]
            data.append((chrom, genotype))

# Creating DataFrame
df = pd.DataFrame(data, columns=['Chromosome', 'Genotype'])

In [5]:
df

,Chromosome,Genotype
0,1,0/0
1,1,0/1
2,1,0/0
3,1,0/0
4,1,./.
...,...,...
595396,MT,0/0
595397,MT,0/0
595398,MT,0/0
595399,MT,0/0


In [6]:
# Function to determine if genotype is heterozygous
def is_heterozygous(gt):
    return gt in ['0/1', '1/0']

# Applying the function and counting heterozygous genotypes
df['Heterozygous'] = df['Genotype'].apply(is_heterozygous)

# Summarizing counts by chromosome
heterozygosity_summary = df.groupby('Chromosome')['Heterozygous'].agg(['sum', 'count'])
heterozygosity_summary.rename(columns={'sum': 'Heterozygous_Count', 'count': 'Total_SNPs'}, inplace=True)
heterozygosity_summary['Heterozygosity_Percent'] = (heterozygosity_summary['Heterozygous_Count'] / heterozygosity_summary['Total_SNPs']) * 100

# Displaying the results
print(heterozygosity_summary)

            Heterozygous_Count  Total_SNPs  Heterozygosity_Percent
Chromosome                                                        
1                        13408       46440               28.871662
10                        8769       29171               30.060677
11                        7898       29040               27.196970
12                        8206       28343               28.952475
13                        6105       21189               28.812119
14                        5406       18638               29.005258
15                        4996       18058               27.666408
16                        5336       19111               27.921093
17                        4624       17956               25.751838
18                        4985       16480               30.248786
19                        3293       12994               25.342466
2                        13480       45743               29.468990
20                        4412       14486               30.45

In [15]:
#у нас есть также не стандртные обозначения генотипа, отфильтруем их
#извлечь все генотипы, кроме стандартных гомозиготных (0/0, 1/1, ./.)
filtered_snps = []

with open(vcf_file, 'r') as file:
    for line in file:
        if not line.startswith('#'):
            parts = line.strip().split('\t')
            genotype = parts[9]
            if genotype not in ('0/0', '1/1', './.'):
                filtered_snps.append(parts)

# Создаем DataFrame из отфильтрованных данных
columns = ['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'GENOTYPE']
filtered_df = pd.DataFrame(filtered_snps, columns=columns)

# Сохраняем в отдельный файл
#filtered_df.to_csv('non_standard_genotypes.csv', sep='\t', index=False)

# Выводим количество отфильтрованных SNP
print(f"Всего отфильтровано SNP (не '0/0', '1/1', './.'): {len(filtered_df)}")



Всего отфильтровано SNP (не '0/0', '1/1', './.'): 164638


In [13]:
filtered_df

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,GENOTYPE
0,1,752721,rs3131972,A,G,.,.,PR,GT,0/1
1,1,798959,rs11240777,A,G,.,.,PR,GT,0/1
2,1,891945,rs13303106,A,G,.,.,PR,GT,0/1
3,1,894573,rs13303010,A,G,.,.,PR,GT,0/1
4,1,903104,rs6696281,C,T,.,.,PR,GT,0/1
...,...,...,...,...,...,...,...,...,...,...
164633,X,155229100,rs731478,C,G,.,.,PR,GT,0/1
164634,X,155229189,rs3093467,C,T,.,.,PR,GT,0/1
164635,X,155229483,rs3093470,C,T,.,.,PR,GT,0/1
164636,X,155232838,rs3093493,A,C,.,.,PR,GT,0/1
